# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

In [189]:
# https://habr.com/ru/search/?target_type=posts&q=python&order_by=date

import requests
from bs4 import BeautifulSoup

# определяем список хабов, которые нам интересны
KEYWORDS = ['python', 'sql']

In [190]:
def get_post_habr(url, keywords):
    data = []
#     req = requests.get('https://habr.com/ru/all/')
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')
    # извлекаем посты
    posts = soup.find_all('article', class_='post')
    for post in posts:
        #   текст поста
        post_text = post.find('div', class_='post__text')
        # если в тексте есть ключевое слово, то будем собирать данные
        if any([word.upper() in post_text.text.upper() for word in keywords]):
            # время статьи
            post_time = post.find('span', class_='post__time')
            _time = post_time.text
            # блок заголовка h2
            post_h2 = post.find('h2', class_='post__title')
            _h2 = post_h2.text
            # берем ссылку из заголовка
            _href = post_h2.find('a').get('href')
            data.append({'time': _time, 'h2': _h2, 'href': _href})
        else:
            continue
    return data

In [191]:
import time
import pandas as pd

posts = pd.DataFrame()

for page in range(50):
    url = 'https://habr.com/ru/all/page' + str(page+1) + '/'
    post = get_post_habr(url, KEYWORDS)
    print(50 - page, end=',')
    if (len(post) > 0):
        posts = pd.concat([posts, pd.DataFrame(post)])  
    time.sleep(0.3)

50,49,48,47,46,45,44,43,42,41,

In [192]:
posts

,time,h2,href
0,сегодня в 20:30,\nAiohttp + Dependency Injector — руководство ...,https://habr.com/ru/post/513510/
0,сегодня в 13:09,\nКак мигрировать Zabbix с MySQL на PostgreSQL...,https://habr.com/ru/post/513470/
1,вчера в 22:51,\nТесты на pytest с генерацией отчетов в Allur...,https://habr.com/ru/post/513432/
0,31 июля 2020 в 16:50,\nКак переписать SQL-запросы на Python с помощ...,https://habr.com/ru/company/plarium/blog/513316/
0,31 июля 2020 в 10:00,\nКак проанализировать рынок фотостудий с помо...,https://habr.com/ru/post/512208/
0,30 июля 2020 в 09:09,\nРабота с базами данных глазами разработчика\n,https://habr.com/ru/company/miro/blog/512990/
1,30 июля 2020 в 08:40,\nО котах и Process Mining\n,https://habr.com/ru/post/513076/
0,29 июля 2020 в 20:55,\nВооруженным глазом: наглядно о проблемах Pos...,https://habr.com/ru/company/tensor/blog/512988/


### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>

## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

In [160]:
import json
import datetime

EMAIL = ['erepkin@gmail.com', 'real1983@mail.ru','amrepkina@mail.ru']

In [162]:
def hack_check(email):
    URL = 'https://digibody.avast.com/v1/web/leaks'
    data = {'email': email}
    req = requests.post(URL, json= data)
    return json.loads(req.text)['value']

In [179]:
info = []

for i in range(len(EMAIL)):
    data = hack_check(EMAIL[i])
    for line in data:
        local_time = time.localtime(line['leak_info']['date']/1000)
        info.append({'Почта': line['username'], 'Дата утечки': time.strftime("%d-%m-%Y", local_time),
                     'Источник утечки': line['leak_info']['title'], 'Описание утечки': line['leak_info']['description']})


bad_email = pd.DataFrame(info)
bad_email
    

,Почта,Дата утечки,Источник утечки,Описание утечки
0,erepkin@gmail.com,22-12-2017,Combolist of 1.4 Billion Credentials,The proliferation of stolen or leaked database...
1,erepkin,23-05-2019,LiveJournal,"In 2017, social network LiveJournal's database..."
2,erepkin@gmail.com,06-02-2019,Collection #4 Combo List,"On January 7, 2019, an online user named Sanix..."
3,erepkin@gmail.com,09-10-2017,Exploit.in combolist,The proliferation of stolen or leaked database...
4,erepkin@gmail.com,06-02-2019,2019 Antipublic Combo List,"On January 7, 2019, an online user named Sanix..."
5,sesamewiss,23-10-2016,Zoosk,"Shortly before the 2016 Memorial Day weekend, ..."
6,0286542305,28-10-2016,Badoo,"In June 2016, millions of accounts compromised..."
7,erepkin@gmail.com,29-01-2019,Collection #2 Combo List,"On January 7, 2019, an online user named Sanix..."
8,erepkin@gmail.com,29-10-2016,VK,Popular Russian social networking platform VKo...
9,erepkin@gmail.com,25-01-2019,Collection #1 Combo List,"On January 7, 2019, an online user named Sanix..."


### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)```
GROUP = 'netology'
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
```

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.

#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий [GitHub](https://github.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам?
Прежде чем задать вопрос необходимо попробовать найти ответ самому в интернете. Навык самостоятельного поиска информации – один из важнейших, и каждый практикующий специалист любого уровня это делает каждый день.

Любой вопрос должен быть сформулирован по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности, прикрепляйте к вопросу скриншоты, либо ссылки на код. Оставляйте только проблемный и воспроизводимый участок кода, все решение выкладывать не допускается.
